## <center>Empirical IO I: Problem Set 0</center>
### <center>Yinan Wang</center>
#### <center>Sept 18, 2020</center>

## Part 0: Logit Inclusive Value
1.If $x_{0} = 0$ , $$IV =\log \sum_{i=0}^N \exp[x_i] = \log (1 + \sum_{i=1}^N \exp[x_i]).$$
Let $f(x) = \log (1 + \sum_{i=1}^N \exp[x_i]),$ 
$$f(tx+(1-t)y) = \log (1 + \sum_{i=1}^N \exp[tx_i+(1-t)y_i]).$$
From Hölder's inequality:
$$\sum_{i=1}^N |x_k y_k| \leq (\sum_{i=1}^N |x_k|^p)^\frac{1}{p} (\sum_{i=1}^N |y_k|^q)^\frac{1}{q}$$
with $$\frac{1}{p} + \frac{1}{q} = 1 $$
Therefore,
$$\sum_{i=1}^N \exp[x_i]^t \exp[y_i]^{1-t} \leq (\sum_{i=1}^N \exp[x_i]^{t{\frac{1}{t}}})^{t} (\sum_{i=1}^N \exp[y_i]^{(1-t){\frac{1}{1-t}}})^{1-t}$$
Thus,
$$\log (\sum_{i=1}^N \exp[x_i]^t \exp[y_i]^{1-t}) \leq t \log (\sum_{i=1}^N \exp[x_i]) +(1-t) \log(\sum_{i=1}^N \exp[y_i])$$
This will also be the case when $x_0 = 0.$

2.Let $m=\max_{i} {x_i},$ then $$IV =\log \sum_{i=0}^N \exp[x_i]=\log \sum_{i=0}^N \exp[x_i-m+m] = \log (\exp[m] \sum_{i=0}^N\exp[x_i-m]) = \log \exp[m] +\log \sum_{i=0}^N\exp[x_i-m] = m+\log \sum_{i=0}^N\exp[x_i-m]$$
Let $IV_m = \log \sum_{i=0}^N\exp[x_i-m]$ be the value after we implementing the trick. Then $IV = IV_m +m$
And implement function as follows:


In [1]:
import numpy as np
def log_sum_exp_trick(a):
    m = max(a)
    a1 = np.subtract(a, m)
    IVm = np.log(np.sum(np.exp(a1)))
    IV = IVm+m
    return IV

3.compare to scipy.misc.logsumexp.Those two results are same. It does not suffer from underflow/overflow errors. 

In [2]:
from scipy.special import logsumexp
a = [900, 930, 960, 980]
IV1 = logsumexp(a)
print('===The result from scipy.misc.logsumexp is: '+str(IV1)+' ===')
IV2 = log_sum_exp_trick(a)
print('===The result from my own function is: '+str(IV2)+' ===')

===The result from scipy.misc.logsumexp is: 980.0000000020611 ===
===The result from my own function is: 980.0000000020611 ===


## Part 1: Markov Chains

In [3]:
def getErgodic(P):
    #The ergodic distribution of P will be an eigenvector of P associated with eigenvalue lamda = 1
    #transpose P so that Markov transitions correspond to right multiplying by a column vector.np.linalg.eig finds right eigenvectors.
    evals, evecs = np.linalg.eig(P.T)
    #find the eigenvector that corresponds to lamda = 1
    evec1 = evecs[:,np.isclose(evals, 1)].real
    #rescale so they sum up to 1
    stationary = evec1 / evec1.sum()
    return stationary[:,0]

P = np.array([[0.2, 0.4, 0.4], [0.1, 0.3, 0.6], [0.5, 0.1, 0.4]])
pi1 = getErgodic(P)
print('===The result from getErgodic is: '+str(pi1)+' ===')
P_100 = np.linalg.matrix_power(P,100)[0]
print('===The result from matrix_power(P, 100) is: '+str(P_100)+' ===')

===The result from getErgodic is: [0.31034483 0.24137931 0.44827586] ===
===The result from matrix_power(P, 100) is: [0.31034483 0.24137931 0.44827586] ===


## Part 2: Numerical Integration
1.Create binomiallogit

In [4]:
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy import integrate

def f(beta,X=0.5):
    return np.exp(np.dot(beta,X)) / (1+np.exp(np.dot(beta,X)))

def binomiallogit(beta, X=0.5, pdf = norm.pdf):
    return f(beta,X)*pdf(beta)

2.Integrate the function using scipy.integrate.quadand set the tolerance to 1e-14

In [5]:
pdf=norm(0.5, 2).pdf
X = 0.5
integrate.quad(binomiallogit, -300, 300,epsabs=1e-14,args = (X,pdf))

(0.5514932726366428, 1.8493264018876166e-09)

3.Integrate the function using Monte Carlo draws of 20 and 400

In [6]:
def montecarlo(n):
    v = np.random.normal(0.5, 2, n)
    fv = np.apply_along_axis(f, 0, v)
    return np.mean(fv)
#20 monte carlo draws
montecarlo(20) #0.5366168183039505
#400 monte carlo draws
montecarlo(400) #0.5418102101803519

0.5517567303024488

4.Integrate the function using Gauss-Hermite quadrature for k = 4,9,12

In [7]:
def cal(x,w,mu,sigma,X):
    y = np.sqrt(2)*sigma*x+mu
    fy = np.apply_along_axis(lambda x:f(x,X=X), 0, y)
    tot = sum(w*fy)
    return (1/np.sqrt(np.pi))*tot



def gaussHermite(k,mu=0.5,sigma=2,X=0.5):
    if k == 4:
        x = np.array([-1.650680123885784555883,
            -0.5246476232752903178841,
            0.5246476232752903178841,
            1.650680123885784555883])
        w = np.array([0.081312835447245177143,
            0.8049140900055128365061,
            0.8049140900055128365061,
            0.08131283544724517714303])
    elif k ==9:
        x = np.array([-3.19099320178152760723,
            -2.266580584531843111802,
            -1.468553289216667931667,
            -0.7235510187528375733226,
            0,
            0.7235510187528375733226,
            1.468553289216667931667,
            2.266580584531843111802,
            3.19099320178152760723])
        w = np.array([3.960697726326438190459e-5,
            0.00494362427553694721722,
            0.088474527394376573288,
            0.4326515590025557501998,
            0.7202352156060509571243,
            0.4326515590025557502,
            0.088474527394376573288,
            0.004943624275536947217225,
            3.96069772632643819046e-5])
    elif k ==12:
        x = np.array([-3.889724897869781919272,
            -3.020637025120889771711,
            -2.279507080501059900188,
            -1.59768263515260479671,
            -0.9477883912401637437046,
            -0.314240376254359111277,
            0.3142403762543591112766,
            0.947788391240163743705,
            1.59768263515260479671,
            2.279507080501059900188,
            3.020637025120889771711,
            3.889724897869781919272])
        w = np.array([2.65855168435630160602e-7,
            8.5736870435878586546e-5,
            0.00390539058462906185999,
            0.05160798561588392999187,
            0.2604923102641611292334,
            0.5701352362624795783471,
            0.5701352362624795783471,
            0.2604923102641611292334,
            0.05160798561588392999187,
            0.00390539058462906185999,
            8.57368704358785865457e-5,
            2.65855168435630160602e-7])
        
    else:
        print('===Not valid option k===')
        return
    return cal(x,w,mu,sigma,X)

gaussHermite(4,mu=0.5,sigma=2,X=0.5) #0.5513138767711852    
# gaussHermite(9,mu=0.5,sigma=2,X=0.5) #0.551494224290647
# gaussHermite(12,mu=0.5,sigma=2,X=0.5) #0.5514932040654398

0.5513138767711852

5.Gauss-Hermite quadrature gives us more accurate result than Monte Carlo does. 

6.Repeat the exercise in two dimensions
$$p(X,\theta) =\int_{-\infty}^{\infty} \int_{-\infty}^{\infty} \frac{\exp(\beta_1*X_1+\beta_2*X_2)}{1+ \exp(\beta_1*X_1+\beta_2*X_2)} f(\boldsymbol{\beta} | \theta) \partial \beta_1  \partial \beta_2$$

In [8]:
def binomiallogit(beta1,beta2, X=[0.5,1], pdf1 = norm.pdf, pdf2=norm.pdf):
    beta = np.array([beta1,beta2])
    return f(beta,X)*pdf1(beta1)*pdf2(beta2)

pdf1 = norm(0.5,2).pdf
pdf2 = norm(1,1).pdf
X=[0.5,1]
integrate.nquad(binomiallogit, [[-100,100], [-100,100]],args = (X,pdf1,pdf2),opts = {'epsabs':1e-14})

(0.7144838053940904, 8.344965756194558e-10)

7.Put everything in two tables.

In [9]:
import pandas as pd
table_1D = pd.DataFrame(columns=['Method','Points','Error'])
new_row = {'Method':'True Value', 'Points':'N/A', 'Error':1e-14}
table_1D = table_1D.append(new_row,ignore_index=True)
new_row = {'Method':'Monte Carlo', 'Points':20, 'Error':1.4876454332692357e-2}
table_1D = table_1D.append(new_row,ignore_index=True)
new_row = {'Method':'Monte Carlo', 'Points':400, 'Error':9.683062456290914e-3}
table_1D = table_1D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':4, 'Error':1.793958654576766e-4}
table_1D = table_1D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':9, 'Error':9.516540041554222e-07}
table_1D = table_1D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':12, 'Error':6.85712030490393e-08}
table_1D = table_1D.append(new_row,ignore_index=True)
table_1D

,Method,Points,Error
0,True Value,N/A,1.000000e-14
1,Monte Carlo,20,1.487645e-02
2,Monte Carlo,400,9.683062e-03
3,Gauss-Hermite,4,1.793959e-04
4,Gauss-Hermite,9,9.516540e-07
5,Gauss-Hermite,12,6.857120e-08


In [10]:
def f2(b1,b2,X=[0.5,1]):
    return np.exp(b1*X[0]+b2*X[1]) / (1+np.exp(b1*X[0]+b2*X[1]))
def montecarlo(n):
    v1 = np.random.normal(0.5, 2, n)
    v2 = np.random.normal(1, 1, n)
    beta = np.array([v1,v2])

    fv = np.apply_along_axis(f2, 0, v1,v2)
    return np.mean(fv)
#20 monte carlo draws
montecarlo(20) #0.7380346232831527
#400 monte carlo draws
montecarlo(400) #0.7143884320073794

0.7083932800177054

In [11]:
# def cal_2D(x,w,k,mu1=0.5,mu2=1,sigma1=2,sigma2=1):
#     to_sum = np.zeros([k,k])
#     for i in range(k):
#         for j in range(k):
#             x1 = np.sqrt(2)*sigma1*x[i]+mu1
#             x2 = np.sqrt(2)*sigma2*x[j]+mu2
#             to_sum[i,j] = w[i]*w[j]*f2(x1, x2)
#     tot = to_sum.sum()
#     return (1/np.pi)*tot

def cal_2D(x,w,k,mu1=0.5,mu2=1,sigma1=2,sigma2=1,X=[0.5,1]):
    # change of variable
    x1 = np.vectorize(lambda x: np.sqrt(2)*sigma1*x+mu1)(x)
    x2 = np.vectorize(lambda x: np.sqrt(2)*sigma2*x+mu2)(x)
    # replicate x1 and x2 to enumertate all pairs
    x1 = np.tile(x1, k).reshape(1,k*k)
    x2 = np.repeat(x2,k).reshape(1,k*k)
    all_pairs = np.concatenate((x1.T, x2.T), axis=1)
    
    # calculate all Vij
    rep_U = all_pairs.dot(np.reshape(X,(2,1))).reshape(k,k)
    fx = np.vectorize(lambda x:np.exp(x) / (1+np.exp(x)))(rep_U)
    # enumertate all pairs of w
    w_all = np.reshape(w,(k,1)).dot(w.reshape(1,k))
    # element-wise mutiplication
    tot = np.multiply(w_all, fx).sum()
    return (1/np.pi)*tot

def gaussHermite_2D(k,mu1=0.5,mu2=1,sigma1=2,sigma2=1,X=0.5):
    if k == 4:
        x = np.array([-1.650680123885784555883,
            -0.5246476232752903178841,
            0.5246476232752903178841,
            1.650680123885784555883])
        w = np.array([0.081312835447245177143,
            0.8049140900055128365061,
            0.8049140900055128365061,
            0.08131283544724517714303])
    elif k ==9:
        x = np.array([-3.19099320178152760723,
            -2.266580584531843111802,
            -1.468553289216667931667,
            -0.7235510187528375733226,
            0,
            0.7235510187528375733226,
            1.468553289216667931667,
            2.266580584531843111802,
            3.19099320178152760723])
        w = np.array([3.960697726326438190459e-5,
            0.00494362427553694721722,
            0.088474527394376573288,
            0.4326515590025557501998,
            0.7202352156060509571243,
            0.4326515590025557502,
            0.088474527394376573288,
            0.004943624275536947217225,
            3.96069772632643819046e-5])
    elif k ==12:
        x = np.array([-3.889724897869781919272,
            -3.020637025120889771711,
            -2.279507080501059900188,
            -1.59768263515260479671,
            -0.9477883912401637437046,
            -0.314240376254359111277,
            0.3142403762543591112766,
            0.947788391240163743705,
            1.59768263515260479671,
            2.279507080501059900188,
            3.020637025120889771711,
            3.889724897869781919272])
        w = np.array([2.65855168435630160602e-7,
            8.5736870435878586546e-5,
            0.00390539058462906185999,
            0.05160798561588392999187,
            0.2604923102641611292334,
            0.5701352362624795783471,
            0.5701352362624795783471,
            0.2604923102641611292334,
            0.05160798561588392999187,
            0.00390539058462906185999,
            8.57368704358785865457e-5,
            2.65855168435630160602e-7])
        
    else:
        print('===Not valid option k===')
        return
    return cal_2D(x,w,k,mu1,mu2,sigma1,sigma2)

# gaussHermite_2D(4) #0.7143713789717683    
# gaussHermite_2D(9) #0.7144837634601805
gaussHermite_2D(12) #0.714483809857691

0.7144838098576911

In [12]:
table_2D = pd.DataFrame(columns=['Method','Points','Error'])
new_row = {'Method':'True Value', 'Points':'N/A', 'Error':1e-14}
table_2D = table_2D.append(new_row,ignore_index=True)
new_row = {'Method':'Monte Carlo', 'Points':20, 'Error':0.023550817889062325}
table_2D = table_2D.append(new_row,ignore_index=True)
new_row = {'Method':'Monte Carlo', 'Points':400, 'Error':9.537338671095075e-05}
table_2D = table_2D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':4, 'Error':0.00011242642232212052}
table_2D = table_2D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':9, 'Error':4.19339099000382e-08}
table_2D = table_2D.append(new_row,ignore_index=True)
new_row = {'Method':'Gauss-Hermite', 'Points':12, 'Error':4.463600600246309e-09}
table_2D = table_2D.append(new_row,ignore_index=True)
table_2D

,Method,Points,Error
0,True Value,N/A,1.000000e-14
1,Monte Carlo,20,2.355082e-02
2,Monte Carlo,400,9.537339e-05
3,Gauss-Hermite,4,1.124264e-04
4,Gauss-Hermite,9,4.193391e-08
5,Gauss-Hermite,12,4.463601e-09


8.Construct binomiallogitmixture that takes a vector of X and returns a vector of binomial probabilities

In [13]:
def binomiallogitmixture(X,mu=0.5,sigma=2,k=12):
    fv = np.vectorize(lambda x:gaussHermite(k,mu,sigma,X=x))(X)
    return fv

binomiallogitmixture([0.6,0.5])

array([0.55805121, 0.5514932 ])